In [105]:
import os
import pandas as pd
import gzip
import ijson
import itertools
import datetime
import json
import os
import re
import urllib.request
from collections import OrderedDict
from bs4 import BeautifulSoup
from lxml import html
from pandas.io.json import json_normalize
import sys

In [106]:
src = os.path.join(os.path.dirname(os.getenv("DATA_DIR")), "src")
sys.path.append(os.path.join(src, "data"))

In [108]:
from extract_text_utils import get_text

### Directories and whatnot

In [82]:
DATA_DIR = os.getenv("DATA_DIR")
content_api = os.path.join(DATA_DIR, "content_api")
content_file = os.path.join(content_api,"content.json.gz")
labelled_file = os.path.join(content_api,"labelled.csv.gz")

In [109]:
count = 0
with gzip.open(content_file, "rt") as reader:
    content_generator = ijson.items(reader, prefix='item')
    for content_item in content_generator:
        count+=1
count

234693

### Extract text, links etc from a `content.json.gz` export (from data science folder) 
Seems suspiciously small

In [110]:
def extract_link_types(content_item):
    links = []
    related_links = []
    coll_links = []
    
    if content_item is not None:
        links = get_text(content_item['details'])
        related_links = []
        coll_links = []
        if 'ordered_related_items' in content_item['links'].keys():
            related_links = [related_item['base_path'] for related_item in
                             content_item['links']['ordered_related_items'] if
                             'base_path' in related_item.keys()]

        if 'documents' in content_item['links'].keys():
            coll_links = [document['base_path'] for document in content_item['links']['documents'] if
                          'base_path' in document.keys()]
        
    return links, related_links, coll_links

In [111]:
destination = os.path.join(content_api, "content_reduced.json.gz")
print("Start:",datetime.datetime.now().strftime("%H:%M:%S"))
with gzip.open(content_file, "rt") as reader, gzip.open(destination, 'wb') as writer:
    content_generator = ijson.items(reader, prefix='item')
    for i,content_item in content_generator:
        row = {}
        row['base_path'] = content_item['base_path']
        row['content_id'] = content_item['content_id']
        row['title'] = content_item['title']
        row['description'] = content_item['description']
        row['details'] = content_item['details']
        l1, l2, l3 = extract_link_types(content_item)
        row['embedded_links'] = l1
        row['related_links'] = l2
        row['coll_links'] = l3
        string_dict = json.dumps(row)
#         row_list.append(string_dict)
        writer.write("{}\n".format(string_dict).encode())
        if i % 10000 == 0:
            print("i:",i,datetime.datetime.now().strftime("%H:%M:%S"))
print("End:",datetime.datetime.now().strftime("%H:%M:%S"))

Start: 03:30:14


ValueError: too many values to unpack (expected 2)

In [ ]:
row_list = []
with gzip.open(destination, 'rt') as reader:
    for line in reader.readlines():
        row_list.append(line)

In [ ]:
df1 = pd.DataFrame([json.loads(s) for s in row_list])

In [49]:
df1.shape

(234693, 8)

In [50]:
df1.head(2)

,base_path,coll_links,content_id,description,details,embdedded_links,related_links,title
0,/aaib-reports/reims-cessna-fa152-g-bgxz-27-may...,[],534b2eeb-505c-41cc-9b2c-eacea6947cd1,,{'attachments': [{'content_id': '4fe1018d-48f9...,[],[],"Reims Cessna FA152, G-BGXZ, 27 May 1988"
1,/employment-tribunal-decisions/mr-k-wieczorek-...,[],7b706877-05e8-4e2c-b061-565ecc3c716c,Employment Tribunal decision.,{'attachments': [{'content_id': '919b1efe-dc62...,[],[],Mr K Wieczorek v Greencore Food to Go Ltd: 332...


In [59]:
destination = os.path.join(content_api, "content_wlinks.csv.gz")

In [61]:
df_wlinks = df1[(df1.embdedded_links.str.len()>0) | (df1.related_links.str.len()>0) | (df1.coll_links.str.len()>0)]
df_wlinks.to_csv(destination, compression="gzip", index=False)

In [ ]:
df1.to_csv(destination, compression="gzip", index=False)

## Set up `src_node` and `dest_node` pairs

In [ ]:
df_wlinks = df1[(df1.embdedded_links.str.len()>0) | (df1.related_links.str.len()>0) | (df1.coll_links.str.len()>0)]

In [63]:
df_wlinks.columns

Index(['base_path', 'coll_links', 'content_id', 'description', 'details',
       'embdedded_links', 'related_links', 'title'],
      dtype='object')

In [74]:
def iterate_links(tupl, ltype_i, ltype_str):
    return [{'src_node': tupl.base_path,
               'dest_node': d,
               'link_type': ltype_str,
               'src_cid': tupl.content_id} for d in tupl[ltype_i]]

In [75]:
link_index_type = [(df_wlinks.columns.get_loc(lt), lt) for lt in ['coll_links', 
                                                                  'embdedded_links', 
                                                                  'related_links']]
link_index_type

[(1, 'coll_links'), (5, 'embdedded_links'), (6, 'related_links')]

In [76]:
rowlist = []
for tup in df_wlinks.itertuples(index=False):
    for lt, ltype in link_index_type:
        list_links = iterate_links(tup, lt, ltype)
        rowlist.extend(list_links)

In [78]:
df_struct_edges = pd.DataFrame(rowlist)

In [79]:
df_struct_edges.head()

,dest_node,link_type,src_cid,src_node
0,/government/uploads/system/uploads/attachment_...,embdedded_links,5ebcfb03-7631-11e4-a3cb-005056011aef,/government/publications/exotic-global-breeds-...
1,/government/publications/wd6-3bl-reviva-compos...,embdedded_links,4f34eed2-02b0-4c26-ba85-d671ee0503f2,/government/publications/wd6-3bl-reviva-compos...
2,/government/uploads/system/uploads/attachment_...,embdedded_links,3b073fb9-ff9c-4164-b71d-f00fa60fc51e,/government/publications/business-incubators-a...
3,/government/uploads/system/uploads/attachment_...,embdedded_links,3b073fb9-ff9c-4164-b71d-f00fa60fc51e,/government/publications/business-incubators-a...
4,/government/uploads/system/uploads/attachment_...,embdedded_links,5c7c9a30-7631-11e4-a3cb-005056011aef,/government/publications/early-morning-tests-a...


### Figure out destination node `content_id`

In [83]:
labelled = pd.read_csv(labelled_file, compression="gzip")
base_cid = dict(zip(labelled.base_path, labelled.content_id))

/Users/felisialoukou/.pyenv/versions/3.6.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [84]:
df_struct_edges['dest_cid'] = df_struct_edges['dest_node'].map(base_cid)

In [88]:
df_struct_edges.shape

(294327, 5)

In [119]:
### Check overlap of structural edges with labelled
diff_label_struct = set(labelled.content_id).difference(set(df_struct_edges.src_cid.values))
len(diff_label_struct)

121232

In [89]:
df_struct_edges[df_struct_edges['dest_cid'].isna()].link_type.value_counts()

embdedded_links    215858
coll_links          13933
related_links         113
Name: link_type, dtype: int64

### Data seems insufficient, might as well extract base_paths in labelled to get all data from the content API

In [120]:
with gzip.open(os.path.join(DATA_DIR, "processed_network", "labelled_for_conext.csv.gz"), "wt") as w:
    w.write("Node\n")
    for tup in labelled.itertuples():
        if tup.content_id in diff_label_struct:
            w.write("{}\n".format(tup.base_path))

In [121]:
temp = pd.read_csv(os.path.join(DATA_DIR, "processed_network", "labelled_for_conext.csv.gz"), compression="gzip")

In [122]:
temp.head()

,Node
0,/aaib-reports/reims-cessna-fa152-g-bgxz-27-may...
1,/aaib-reports/pterodactyl-microlight-g-mboj-6-...
2,/aaib-reports/aar-4-2007-airbus-a340-642-g-vat...
3,/aaib-reports/enstrom-280c-uk-g-ctsi-14-januar...
4,/aaib-reports/aaib-investigation-to-pitts-supe...


In [129]:
temp.shape[0]/2

87401.0

In [130]:
temp[0:87000].to_csv(os.path.join(DATA_DIR, "processed_network", "labelled_for_conext_pt1.csv.gz"), 
                      compression="gzip", index=False)

In [131]:
temp[87000:].to_csv(os.path.join(DATA_DIR, "processed_network", "labelled_for_conext_pt2.csv.gz"), 
                      compression="gzip", index=False)